In [9]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [13]:
df = pd.read_csv("fashion-mnist_train.csv")

df.head()

labels = df.label
X = df.drop(["label"], axis = 1)
labels = labels.to_numpy()

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)

print(f"Shape of train set is {X.shape}")
print(f"Shape of train label is {labels.shape}")

y_label = np.zeros([60000, 10])

for i in range(len(labels)):
    y_label[i][labels[i]]= 1

Shape of train set is (60000, 784)
Shape of train label is (60000,)


In [11]:
class NeuralNet():
    def __init__(self, layers=[784, 398, 10], alpha = 0.01, num_iters = 100):
        self.params = {}
        self.layers = layers
        self.alpha= alpha
        self.num_iters = num_iters
        self.loss = []
        self.sample_size = None
        self.X = None
        self.y = None
        
    def init_weights(self):
        np.random.seed(1)
        self.params["W1"] = np.random.randn(self.layers[0], self.layers[1]) 
        self.params['b1']  =np.random.randn(self.layers[1],)
        self.params['W2'] = np.random.randn(self.layers[1],self.layers[2]) 
        self.params['b2'] = np.random.randn(self.layers[2],)
        
    def relu(self,Z):
        return np.maximum(0,Z)
    
    def sigmoid(self,Z):
        np.seterr(over = 'ignore')
        return 1.0 / (1.0 + np.exp(-Z))


    def entropy_loss(self,y, h):
        nsample = len(y)
        loss = -1/nsample * (np.sum(np.multiply(np.log(h), y) + np.multiply((1 - y), np.log(1 - h))))
        return loss

    
    
    
    
    def forward_propagation(self):
        '''
        Performs the forward propagation
        '''
        
        Z1 = self.X.dot(self.params['W1']) + self.params['b1']
        A1 = self.relu(Z1)
        Z2 = A1.dot(self.params['W2']) + self.params['b2']
        h = self.sigmoid(Z2)
        loss = self.entropy_loss(self.y,h)

        # save calculated parameters     
        self.params['Z1'] = Z1
        self.params['Z2'] = Z2
        self.params['A1'] = A1
        
        return h, loss
    
    def back_propagation(self,yhat):
        def dRelu(x):
            x[x<=0] = 0
            x[x>0] = 1
            return x
        
        dl_wrt_yhat = -(np.divide(self.y,yhat) - np.divide((1 - self.y),(1-h)))
        dl_wrt_sig = yhat * (1-h)
        dl_wrt_z2 = dl_wrt_yhat * dl_wrt_sig

        dl_wrt_A1 = dl_wrt_z2.dot(self.params['W2'].T)
        dl_wrt_w2 = self.params['A1'].T.dot(dl_wrt_z2)
        dl_wrt_b2 = np.sum(dl_wrt_z2, axis=0)

        dl_wrt_z1 = dl_wrt_A1 * dRelu(self.params['Z1'])
        dl_wrt_w1 = self.X.T.dot(dl_wrt_z1)
        dl_wrt_b1 = np.sum(dl_wrt_z1, axis=0)

        #update the weights and bias
        self.params['W1'] = self.params['W1'] - self.alpha * dl_wrt_w1
        self.params['W2'] = self.params['W2'] - self.alpha * dl_wrt_w2
        self.params['b1'] = self.params['b1'] - self.alpha * dl_wrt_b1
        self.params['b2'] = self.params['b2'] - self.alpha * dl_wrt_b2
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        self.init_weights() #initialize weights and bias


        for i in range(self.num_iters):
            h, loss = self.forward_propagation()
            self.back_propagation(h)
            self.loss.append(loss)
            
            
    def predict(self, X):
        Z1 = X.dot(self.params['W1']) + self.params['b1']
        A1 = self.relu(Z1)
        Z2 = A1.dot(self.params['W2']) + self.params['b2']
        pred = self.sigmoid(Z2)
        return np.round(pred) 
    
    def acc(self, y, h):
        acc = int(sum(y == h) / len(y) * 100)
        return acc


    def plot_loss(self):
        '''
        Plots the loss curve
        '''
        plt.plot(self.loss)
        plt.xlabel("Iteration")
        plt.ylabel("logloss")
        plt.title("Loss curve for training")
        plt.show()
        

In [12]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y_label, test_size=0.2, random_state=2)


nn = NeuralNet()
nn.fit(Xtrain, ytrain)
nn.plot_loss()

<ipython-input-11-cdd630db8910>:29: RuntimeWarning: divide by zero encountered in log
  loss = -1/nsample * (np.sum(np.multiply(np.log(h), y) + np.multiply((1 - y), np.log(1 - h))))
<ipython-input-11-cdd630db8910>:29: RuntimeWarning: invalid value encountered in multiply
  loss = -1/nsample * (np.sum(np.multiply(np.log(h), y) + np.multiply((1 - y), np.log(1 - h))))
<ipython-input-11-cdd630db8910>:60: RuntimeWarning: divide by zero encountered in true_divide
  dl_wrt_yhat = -(np.divide(self.y,yhat) - np.divide((1 - self.y),(1-h)))
<ipython-input-11-cdd630db8910>:60: RuntimeWarning: invalid value encountered in true_divide
  dl_wrt_yhat = -(np.divide(self.y,yhat) - np.divide((1 - self.y),(1-h)))


NameError: name 'h' is not defined